## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [59]:
# Load up store_income_data.csv
import pandas as pd
import fuzzywuzzy
from fuzzywuzzy import process
import datetime
import chardet

# Load up CSV file
income_df = pd.read_csv("store_income_data_task.csv")


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [75]:
income_df.head(10)

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,united kingdom
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,united kingdom
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,united kingdom
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,united kingdom
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,united kingdom
5,6,"Auburn National Bancorporation, Inc.",ccaldeyroux5@dion.ne.jp,Grocery,$69798987.04,19-9-1999,u.k.
6,7,"Interlink Electronics, Inc.",orodenborch6@skyrock.com,Garden,$22521052.79,8-6-2001,sa
7,8,"Tallgrass Energy Partners, LP",NaN,Grocery,$54405380.40,16-9-1992,u.k/
8,9,Tronox Limited,NaN,Outdoors,$99290004.13,11-1-1992,united kingdom
9,10,"Synopsys, Inc.",lcancellieri9@tmall.com,Electronics,$44091294.62,11-7-2006,united kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [76]:
# Display unique country names before cleaning
countries = income_df['country'].unique()
print(f"Number of unique countries before cleaning: {len(countries)}")
countries

# Convert country names to lowercase and remove leading/trailing whitespaces
income_df['country'] = income_df['country'].str.lower().str.strip()

# Replace all null/NaN value country names with 'unknown'
income_df.country.fillna('unknown', inplace=True)

# Display unique country names after cleaning
countries = income_df['country'].unique()
print(f"Number of unique countries after cleaning: {len(countries)}")
countries

Number of unique countries before cleaning: 14
Number of unique countries after cleaning: 14


C:\Users\User\AppData\Local\Temp\ipykernel_30076\3732283081.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  income_df.country.fillna('unknown', inplace=True)


array(['united kingdom', 'u.k.', 'sa', 'u.k/', 'unknown', 'south africa',
       's.a', 'u.k', 'united states', 'sa.', 'sa/',
       's. africasouth africa', 's. africasouth africa/',
       's. africasouth africa.'], dtype=object)

In [77]:
matches = fuzzywuzzy.process.extract("s.a", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('s.a', 100),
 ('sa', 40),
 ('south africa', 40),
 ('sa.', 40),
 ('sa/', 40),
 ('u.k.', 33),
 ('u.k/', 33),
 ('u.k', 33),
 ('s. africasouth africa', 26),
 ('s. africasouth africa/', 26)]

In [79]:
def replace_country_names(df, column, string_to_match, min_ratio):
    """
    Replaces country names that are a close match using fuzzy logic and a custom ratio

    Arguments:  df - the dataframe to search
                column - the column to search
                string_to_match - the string to search for
                ratio - the minimum ratio to consider a match
    """

    # Get a list of unique names for the specified column
    unique_strings = df[column].unique()

    # Get the top 10 closest matches for the string_to_match value
    matches = fuzzywuzzy.process.extract(string_to_match, unique_strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio greater than or equal to min_ratio value
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

     # Let us know when the function is done
    print("Replacement completed")


# The min ratios were determined in the previous code cell by analyzing the relevant
# top 10 closest matches for each string_to_match value
replace_country_names(df=income_df, column='country', string_to_match="uk", min_ratio=40)
replace_country_names(df=income_df, column='country', string_to_match="united kingdom", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="britain", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="england", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="united states", min_ratio=70)
replace_country_names(df=income_df, column='country', string_to_match="america", min_ratio=90)
replace_country_names(df=income_df, column='country', string_to_match="south africa", min_ratio=75)
replace_country_names(df=income_df, column='country', string_to_match="s.a.", min_ratio=40)


Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed
Replacement completed


In [80]:
def replace_name(string_to_find, new_name):
    """
    Replaces the specified country name with a specified new name

    Arguments:  string_to_find - country name to be replaced
                new_name - name to replace the old name with
    """
    income_df.replace(string_to_find, new_name, inplace=True)

    # Let us know when the function is done
    print("Name replaced")


replace_name(string_to_find='united states', new_name="United States")
replace_name(string_to_find='america', new_name="United States")
replace_name(string_to_find='s.a.', new_name="South Africa")
replace_name(string_to_find='britain', new_name="United Kingdom")
replace_name(string_to_find='united kingdom', new_name="United Kingdom")
replace_name(string_to_find='uk', new_name="United Kingdom")
replace_name(string_to_find='england', new_name="United Kingdom")
replace_name(string_to_find='/', new_name="unknown")
replace_name(string_to_find='.', new_name="unknown")
replace_name(string_to_find='', new_name="unknown")

# Display unique country names after all cleaning
countries = income_df['country'].unique()
print(f"\nNumber of unique countries: {len(countries)}")
countries

Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced
Name replaced

Number of unique countries: 3


array(['United Kingdom', 'South Africa', 'United States'], dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [81]:
# modules we'll use
from datetime import date
# print the first few rows of the date column
income_df.date_measured.head()

0     4-2-2006
1     4-1-2006
2    12-9-2003
3     8-5-2006
4    21-1-1973
Name: date_measured, dtype: object

In [72]:
# check the data type of our date column
income_df['date_measured'].dtype

dtype('O')

In [85]:
# create a new column, days_ago, with the parsed dates
income_df['date_measured'] = pd.to_datetime(income_df['date_measured'], format='%d-%m-%Y')

income_df['days_ago'] = pd.to_datetime(income_df['date_measured'], format='%d %B %Y')


0   2006-02-04
1   2006-01-04
2   2003-09-12
3   2006-05-08
4   1973-01-21
Name: days_ago, dtype: datetime64[ns]

In [84]:
# current date and time
from datetime import datetime
today = datetime.now()
print ('Current date and time : ', today)

Current date and time :  2024-04-24 13:21:37.504630


In [86]:
#income_df.head(20)
income_df['days_ago'].head()

0   2006-02-04
1   2006-01-04
2   2003-09-12
3   2006-05-08
4   1973-01-21
Name: days_ago, dtype: datetime64[ns]

In [87]:
countries = income_df['country'].unique()
countries

array(['United Kingdom', 'South Africa', 'United States'], dtype=object)